![banner](./images/knobs.jpeg)

# Recommendation System for DJ'ing Electronic Music
***
Author: [Paul Lindquist](https://www.linkedin.com/in/paul-lindquist/)

## Overview
***
TBD

## Objective
***
TBD

## Data
***
TBD

## Methods
***
TBD

## Library Imports
***

In [1]:
# Libraries and packages
import pandas as pd
from tqdm import tqdm
import time
import requests

# Spotify authorization .py
import spotify_authorization

## Manual API Calls to Spotify Web API
***
Registered a Spotify Developer account to get a Client ID and Client Secret. Used this to browse genres within Spotify's "categories."

In [2]:
CLIENT_ID = '4eb9684aba474efa92f55be38917497e'
CLIENT_SECRET = '80f447a4ac9a40ef99dac84fc345e3b9'

AUTH_URL = 'https://accounts.spotify.com/api/token'

auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

auth_response_data = auth_response.json()

access_token = auth_response_data['access_token']

headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

BASE_URL = 'https://api.spotify.com/v1/'

categories = 'browse/categories'

r = requests.get(BASE_URL + categories, headers=headers)

r = r.json()
r

{'categories': {'href': 'https://api.spotify.com/v1/browse/categories?offset=0&limit=20',
  'items': [{'href': 'https://api.spotify.com/v1/browse/categories/toplists',
    'icons': [{'height': 275,
      'url': 'https://t.scdn.co/media/derived/toplists_11160599e6a04ac5d6f2757f5511778f_0_0_275_275.jpg',
      'width': 275}],
    'id': 'toplists',
    'name': 'Top Lists'},
   {'href': 'https://api.spotify.com/v1/browse/categories/hiphop',
    'icons': [{'height': 274,
      'url': 'https://t.scdn.co/media/original/hip-274_0a661854d61e29eace5fe63f73495e68_274x274.jpg',
      'width': 274}],
    'id': 'hiphop',
    'name': 'Hip-Hop'},
   {'href': 'https://api.spotify.com/v1/browse/categories/pop',
    'icons': [{'height': 274,
      'url': 'https://t.scdn.co/media/derived/pop-274x274_447148649685019f5e2a03a39e78ba52_0_0_274_274.jpg',
      'width': 274}],
    'id': 'pop',
    'name': 'Pop'},
   {'href': 'https://api.spotify.com/v1/browse/categories/country',
    'icons': [{'height': None,


In [3]:
BASE_URL = 'https://api.spotify.com/v1/'

dj_music = 'browse/categories/edm_dance'

r = requests.get(BASE_URL + dj_music, headers=headers)

r = r.json()
r

{'href': 'https://api.spotify.com/v1/browse/categories/edm_dance',
 'icons': [{'height': 274,
   'url': 'https://t.scdn.co/media/derived/edm-274x274_0ef612604200a9c14995432994455a6d_0_0_274_274.jpg',
   'width': 274}],
 'id': 'edm_dance',
 'name': 'Dance/Electronic'}

## Data Imports, Creation, Concatenation and Cleaning
***

### Spotify API Calls

In [4]:
# Authorize Spotify API access
sp = spotify_authorization.authorize()

In [5]:
# Get a list of all Spotify genres to narrow down to electronic music-specific
print(sp.recommendation_genre_seeds())

['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient', 'anime', 'black-metal', 'bluegrass', 'blues', 'bossanova', 'brazil', 'breakbeat', 'british', 'cantopop', 'chicago-house', 'children', 'chill', 'classical', 'club', 'comedy', 'country', 'dance', 'dancehall', 'death-metal', 'deep-house', 'detroit-techno', 'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm', 'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk', 'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove', 'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle', 'heavy-metal', 'hip-hop', 'holidays', 'honky-tonk', 'house', 'idm', 'indian', 'indie', 'indie-pop', 'industrial', 'iranian', 'j-dance', 'j-idol', 'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino', 'malay', 'mandopop', 'metal', 'metal-misc', 'metalcore', 'minimal-techno', 'movies', 'mpb', 'new-age', 'new-release', 'opera', 'pagode', 'party', 'philippines-opm', 'piano', 'pop', 'pop-film', 'post-dubstep', 'power-po

In [6]:
# Set genres for electronic music
genres = ['breakbeat', 'chicago-house', 'club', 'dance', 'dancehall',
          'deep-house', 'detroit-techno', 'disco', 'drum-and-bass', 'dub',
          'dubstep', 'edm', 'electro', 'electronic', 'funk', 'garage',
          'hardstyle', 'house', 'idm', 'indie-pop', 'industrial', 'j-dance',
          'minimal-techo', 'new-release', 'party', 'post-dubstep',
          'progressive-house', 'synth-pop', 'techno', 'trance', 'trip-hop',
          'work-out']

Write a function that creates a dictionary of 100 songs for each designated genre listed above

In [7]:
def spotify_dataset_by_genres(genre_list):
    
    # Set number of recommendations per genre (100 max according to Spotify
    # API documentation)
    n_recs = 100
    
    # Create a dictionary with desired info to extract from each genre
    data_dict = {'artist_name': [], 'track_name': [], 'genre': [], 'tempo': [],
                 'danceability': [], 'energy': [], 'valence': [], 'id': []}
    
    # Get recs for every genre
    for g in genre_list:
        
        # Get n recommendations
        recs = sp.recommendations(genres=[g], limit=n_recs)

        # JSON-like string to dictionary
        recs = eval(recs.json().replace('null', '-999').replace('false', 'False').replace('true', 'True'))['tracks']

        # Iterate through & extract data from each track
        for track in recs:

            # Metadata
            track_meta = sp.track(track['id'])
            data_dict['track_name'].append(track_meta.name)
            data_dict['artist_name'].append(track_meta.album.artists[0].name)

            # ID and genre
            data_dict['genre'].append(g)
            data_dict['id'].append(track['id'])

            # Track features
            track_features = sp.track_audio_features(track['id'])
            data_dict['tempo'].append(track_features.tempo)
            data_dict['danceability'].append(track_features.danceability)
            data_dict['energy'].append(track_features.energy)
            data_dict['valence'].append(track_features.valence)

            # Wait .2 seconds per track so as not to overload API
            time.sleep(0.2)

    # Store data in DataFrame
    df = pd.DataFrame(data_dict)

    # Drop duplicates
    df.drop_duplicates(subset='id', keep='first', inplace=True)

    return df

### Dataset Build-Out

#### *Below is a sample function call used to build datasets. I kept getting myriad of ReadTimeout, Bad Gateway, etc. errors related to the API calls during multiple DataFrame creation so I unfortanetly had to resort to doing so manually, one at a time.*

In [ ]:
spotify_1 = spotify_dataset_by_genres(genres)

#### *The code blocks below were used to manually create datasets. Due to being computationally expensive, they've been commented out.*

In [8]:
# spotify_2 = spotify_dataset_by_genres(genres)

In [9]:
# spotify_3 = spotify_dataset_by_genres(genres)

In [10]:
# spotify_1_thru_3 = pd.concat([spotify_1, spotify_2, spotify_3],
#                             ignore_index=True)

In [11]:
# spotify_1_thru_3 = spotify_1_thru_3.drop_duplicates(subset='id', keep='first')

In [12]:
# spotify_1_thru_3.to_csv('spotify_1_thru_3.csv', index=False)

In [13]:
# spotify_4 = spotify_dataset_by_genres(genres)

In [14]:
# spotify_5 = spotify_dataset_by_genres(genres)

In [15]:
# spotify_6 = spotify_dataset_by_genres(genres)

In [16]:
# spotify_4_thru_6 = pd.concat([spotify_4, spotify_5, spotify_6],
#                             ignore_index=True)
# spotify_4_thru_6 = spotify_4_thru_6.drop_duplicates(subset='id', keep='first')
# spotify_4_thru_6.to_csv('spotify_4_thru_6.csv', index=False)

In [17]:
# spotify_7 = spotify_dataset_by_genres(genres)

In [18]:
# spotify_8 = spotify_dataset_by_genres(genres)

In [19]:
# spotify_9 = spotify_dataset_by_genres(genres)

In [20]:
# spotify_7_thru_9 = pd.concat([spotify_7, spotify_8, spotify_9],
#                             ignore_index=True)
# spotify_7_thru_9 = spotify_7_thru_9.drop_duplicates(subset='id', keep='first')
# spotify_7_thru_9.to_csv('spotify_7_thru_9.csv', index=False)

In [21]:
# spotify_10 = spotify_dataset_by_genres(genres)

In [22]:
# spotify_11 = spotify_dataset_by_genres(genres)

In [23]:
# spotify_12 = spotify_dataset_by_genres(genres)

In [24]:
# spotify_10_thru_12 = pd.concat([spotify_10, spotify_11, spotify_12],
#                             ignore_index=True)
# spotify_10_thru_12 = spotify_10_thru_12.drop_duplicates(subset='id', keep='first')
# spotify_10_thru_12.to_csv('spotify_10_thru_12.csv', index=False)

In [25]:
# spotify_13 = spotify_dataset_by_genres(genres)

In [26]:
# spotify_14 = spotify_dataset_by_genres(genres)

In [27]:
# spotify_13_thru_14 = pd.concat([spotify_13, spotify_14],
#                             ignore_index=True)
# spotify_13_thru_14 = spotify_13_thru_14.drop_duplicates(subset='id', keep='first')
# spotify_13_thru_14.to_csv('spotify_13_thru_14.csv', index=False)

### Dataset Concatenation

In [58]:
genres_set_1 = pd.read_csv('./data/spotify_1_thru_3.csv')
genres_set_2 = pd.read_csv('./data/spotify_4_thru_6.csv')
genres_set_3 = pd.read_csv('./data/spotify_7_thru_9.csv')
genres_set_4 = pd.read_csv('./data/spotify_10_thru_12.csv')
genres_set_5 = pd.read_csv('./data/spotify_13_thru_14.csv')
sp_more_genres = pd.read_csv('./data/songs_all_genres_trunc.csv')

In [59]:
sp_recommended = pd.concat([genres_set_1, genres_set_2,
                           genres_set_3, genres_set_4,
                           genres_set_5], ignore_index=True)
sp_recommended

,artist_name,track_name,genre,tempo,danceability,energy,valence,id
0,Various Artists,Dun Proppa,breakbeat,103.995,0.651,0.977,0.781,6a5aQlAAdgObJ3KhM9NRZW
1,Various Artists,Who's Making Love,breakbeat,102.052,0.709,0.390,0.815,42UHQGcpq8oxAujCQpxGII
2,The Future Sound Of London,Papua New Guinea,breakbeat,124.870,0.550,0.575,0.371,1r1OeKQ1y7Q20A6ekUXtYq
3,Various Artists,"Don't Call Me Nigger, Whitey",breakbeat,183.978,0.602,0.921,0.512,56r13tQH07EPshIRvW9eXy
4,DJ Andy Smith,The Choice Is Yours,breakbeat,101.397,0.879,0.847,0.711,6281LdA2cVFwVM0xC5cqA8
...,...,...,...,...,...,...,...,...
26034,Busta Rhymes,Break Ya Neck,work-out,83.011,0.820,0.809,0.737,0svl7cK07gK1ia5ainczU5
26035,Eminem,Without Me,work-out,112.230,0.919,0.657,0.659,5pBvNeOAJ54zgd5lEOmM3b
26036,Destiny's Child,Survivor,work-out,161.109,0.514,0.911,0.619,2Mpj1Ul5OFPyyP4wB62Rvi
26037,Bloc Party,Helicopter,work-out,170.058,0.466,0.954,0.396,72aCha2Meq7txnk451hz7k


In [60]:
sp_recommended.rename(columns={'artist_name': 'Artists',
                               'track_name': 'Song_Title',
                               'genre': 'Genre',
                               'tempo': 'Tempo',
                               'danceability': 'Danceability',
                               'energy': 'Energy',
                               'valence': 'Valence',
                               'id': 'ID'}, inplace=True)
sp_recommended

,Artists,Song_Title,Genre,Tempo,Danceability,Energy,Valence,ID
0,Various Artists,Dun Proppa,breakbeat,103.995,0.651,0.977,0.781,6a5aQlAAdgObJ3KhM9NRZW
1,Various Artists,Who's Making Love,breakbeat,102.052,0.709,0.390,0.815,42UHQGcpq8oxAujCQpxGII
2,The Future Sound Of London,Papua New Guinea,breakbeat,124.870,0.550,0.575,0.371,1r1OeKQ1y7Q20A6ekUXtYq
3,Various Artists,"Don't Call Me Nigger, Whitey",breakbeat,183.978,0.602,0.921,0.512,56r13tQH07EPshIRvW9eXy
4,DJ Andy Smith,The Choice Is Yours,breakbeat,101.397,0.879,0.847,0.711,6281LdA2cVFwVM0xC5cqA8
...,...,...,...,...,...,...,...,...
26034,Busta Rhymes,Break Ya Neck,work-out,83.011,0.820,0.809,0.737,0svl7cK07gK1ia5ainczU5
26035,Eminem,Without Me,work-out,112.230,0.919,0.657,0.659,5pBvNeOAJ54zgd5lEOmM3b
26036,Destiny's Child,Survivor,work-out,161.109,0.514,0.911,0.619,2Mpj1Ul5OFPyyP4wB62Rvi
26037,Bloc Party,Helicopter,work-out,170.058,0.466,0.954,0.396,72aCha2Meq7txnk451hz7k


In [61]:
sp_more_genres

,Name,Id,Artists,Artists_Id,Release,Duration,Genre,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valeance,Tempo
0,Hero,29p6UW08MXzwj9thWBW3uF,"['Afrojack', 'David Guetta']","['4D75GcNG95ebPtNvoNVXhz', '1Cs0zKBU1kc0i8ypK3...",4/30/21,171840,edm,0.713,0.772,2,-5.032,0.0313,0.157000,0.0000,0.1530,0.5230,125.001
1,Chasing Stars (feat. James Bay),6y6xhAgZjvxy5kR5rigpY3,"['Alesso', 'Marshmello', 'James Bay']","['4AVFqumd2ogHFlRbKIjp1t', '64KEffDW9EtZ1y2vBY...",8/20/21,170457,edm,0.594,0.640,10,-6.865,0.0330,0.016900,0.0000,0.1350,0.4920,92.871
2,Gotta Be You,2Q8fFORnsDE6I3RNvSLXSf,"['NERVO', 'Carla Monroe']","['4j5KBTO4tk7up54ZirNGvK', '4S9LNSZusH3XflT3g3...",1/8/21,149032,edm,0.642,0.869,6,-5.271,0.0452,0.252000,0.0288,0.0708,0.3760,123.910
3,Okay,71H8k9qe7DersxR6KyhUnI,"['Nicky Romero', 'MARF', 'Wulf']","['5ChF3i92IPZHduM7jN3dpg', '6y1eDna5tYFgcvKyGh...",7/23/21,151312,edm,0.513,0.836,2,-3.781,0.0579,0.046300,0.0000,0.1280,0.3040,124.964
4,Get Low,3oZoXyU0SkDldgS7AcN4y4,"['Dillon Francis', 'DJ Snake']","['5R3Hr2cnCCjt220Jmt2xLf', '540vIaP2JwjQb9dm3a...",10/22/14,212773,edm,0.674,0.932,11,-4.424,0.0859,0.028200,0.7100,0.6840,0.7280,101.033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4162,Limba,4bGh9ITVoiSTrtLg0h1PQo,"['Einmusik', 'Seth Schwarz']","['1LXTXZjheh25pXMEUT9iC1', '0zUt9PboFsELRcvO7U...",10/9/20,472379,deep house,0.703,0.732,1,-10.206,0.0436,0.000955,0.8450,0.2340,0.1830,123.989
4163,Can I,3qavQewlgidBuYHrJe3Qqb,['Britta Arnold'],['7JpBTUqubY86KynSJr28RN'],3/19/21,511990,deep house,0.809,0.667,7,-7.797,0.0565,0.000623,0.8330,0.0920,0.0325,120.019
4164,By Defection,2auhKxSJCfnsaOVxFoF3uA,['Dixie Yure'],['63Hs70Er0FXT7oVFaiP3Xb'],4/27/10,506468,deep house,0.927,0.513,5,-8.091,0.0646,0.011700,0.7900,0.1680,0.2420,124.019
4165,Chunky,5SB73InpxuEQ20qEWhOQFV,['Format:B'],['5Am25tT39BPzreHngMkuux'],10/23/15,158115,deep house,0.803,0.858,10,-4.019,0.0538,0.011100,0.0960,0.0766,0.8620,121.989


### Data Cleaning

In [62]:
cols_order = ['Song_Title', 'Artists', 'Genre', 'Tempo', 'Danceability',
            'Energy', 'Valence', 'ID']

In [63]:
sp_recommended = sp_recommended[cols_order]
sp_recommended

,Song_Title,Artists,Genre,Tempo,Danceability,Energy,Valence,ID
0,Dun Proppa,Various Artists,breakbeat,103.995,0.651,0.977,0.781,6a5aQlAAdgObJ3KhM9NRZW
1,Who's Making Love,Various Artists,breakbeat,102.052,0.709,0.390,0.815,42UHQGcpq8oxAujCQpxGII
2,Papua New Guinea,The Future Sound Of London,breakbeat,124.870,0.550,0.575,0.371,1r1OeKQ1y7Q20A6ekUXtYq
3,"Don't Call Me Nigger, Whitey",Various Artists,breakbeat,183.978,0.602,0.921,0.512,56r13tQH07EPshIRvW9eXy
4,The Choice Is Yours,DJ Andy Smith,breakbeat,101.397,0.879,0.847,0.711,6281LdA2cVFwVM0xC5cqA8
...,...,...,...,...,...,...,...,...
26034,Break Ya Neck,Busta Rhymes,work-out,83.011,0.820,0.809,0.737,0svl7cK07gK1ia5ainczU5
26035,Without Me,Eminem,work-out,112.230,0.919,0.657,0.659,5pBvNeOAJ54zgd5lEOmM3b
26036,Survivor,Destiny's Child,work-out,161.109,0.514,0.911,0.619,2Mpj1Ul5OFPyyP4wB62Rvi
26037,Helicopter,Bloc Party,work-out,170.058,0.466,0.954,0.396,72aCha2Meq7txnk451hz7k


In [64]:
drop_cols = ['Artists_Id', 'Release', 'Duration', 'Key', 'Loudness',
            'Speechiness', 'Acousticness', 'Instrumentalness',
            'Liveness']

In [65]:
sp_more_genres.drop(columns=drop_cols, inplace=True)
sp_more_genres

,Name,Id,Artists,Genre,Danceability,Energy,Valeance,Tempo
0,Hero,29p6UW08MXzwj9thWBW3uF,"['Afrojack', 'David Guetta']",edm,0.713,0.772,0.5230,125.001
1,Chasing Stars (feat. James Bay),6y6xhAgZjvxy5kR5rigpY3,"['Alesso', 'Marshmello', 'James Bay']",edm,0.594,0.640,0.4920,92.871
2,Gotta Be You,2Q8fFORnsDE6I3RNvSLXSf,"['NERVO', 'Carla Monroe']",edm,0.642,0.869,0.3760,123.910
3,Okay,71H8k9qe7DersxR6KyhUnI,"['Nicky Romero', 'MARF', 'Wulf']",edm,0.513,0.836,0.3040,124.964
4,Get Low,3oZoXyU0SkDldgS7AcN4y4,"['Dillon Francis', 'DJ Snake']",edm,0.674,0.932,0.7280,101.033
...,...,...,...,...,...,...,...,...
4162,Limba,4bGh9ITVoiSTrtLg0h1PQo,"['Einmusik', 'Seth Schwarz']",deep house,0.703,0.732,0.1830,123.989
4163,Can I,3qavQewlgidBuYHrJe3Qqb,['Britta Arnold'],deep house,0.809,0.667,0.0325,120.019
4164,By Defection,2auhKxSJCfnsaOVxFoF3uA,['Dixie Yure'],deep house,0.927,0.513,0.2420,124.019
4165,Chunky,5SB73InpxuEQ20qEWhOQFV,['Format:B'],deep house,0.803,0.858,0.8620,121.989


In [66]:
sp_more_genres.rename(columns={'Name': 'Song_Title', 'Id': 'ID',
                               'Valeance': 'Valence'}, inplace=True)
sp_more_genres

,Song_Title,ID,Artists,Genre,Danceability,Energy,Valence,Tempo
0,Hero,29p6UW08MXzwj9thWBW3uF,"['Afrojack', 'David Guetta']",edm,0.713,0.772,0.5230,125.001
1,Chasing Stars (feat. James Bay),6y6xhAgZjvxy5kR5rigpY3,"['Alesso', 'Marshmello', 'James Bay']",edm,0.594,0.640,0.4920,92.871
2,Gotta Be You,2Q8fFORnsDE6I3RNvSLXSf,"['NERVO', 'Carla Monroe']",edm,0.642,0.869,0.3760,123.910
3,Okay,71H8k9qe7DersxR6KyhUnI,"['Nicky Romero', 'MARF', 'Wulf']",edm,0.513,0.836,0.3040,124.964
4,Get Low,3oZoXyU0SkDldgS7AcN4y4,"['Dillon Francis', 'DJ Snake']",edm,0.674,0.932,0.7280,101.033
...,...,...,...,...,...,...,...,...
4162,Limba,4bGh9ITVoiSTrtLg0h1PQo,"['Einmusik', 'Seth Schwarz']",deep house,0.703,0.732,0.1830,123.989
4163,Can I,3qavQewlgidBuYHrJe3Qqb,['Britta Arnold'],deep house,0.809,0.667,0.0325,120.019
4164,By Defection,2auhKxSJCfnsaOVxFoF3uA,['Dixie Yure'],deep house,0.927,0.513,0.2420,124.019
4165,Chunky,5SB73InpxuEQ20qEWhOQFV,['Format:B'],deep house,0.803,0.858,0.8620,121.989


In [67]:
sp_more_genres = sp_more_genres[cols_order]

sp_more_genres

,Song_Title,Artists,Genre,Tempo,Danceability,Energy,Valence,ID
0,Hero,"['Afrojack', 'David Guetta']",edm,125.001,0.713,0.772,0.5230,29p6UW08MXzwj9thWBW3uF
1,Chasing Stars (feat. James Bay),"['Alesso', 'Marshmello', 'James Bay']",edm,92.871,0.594,0.640,0.4920,6y6xhAgZjvxy5kR5rigpY3
2,Gotta Be You,"['NERVO', 'Carla Monroe']",edm,123.910,0.642,0.869,0.3760,2Q8fFORnsDE6I3RNvSLXSf
3,Okay,"['Nicky Romero', 'MARF', 'Wulf']",edm,124.964,0.513,0.836,0.3040,71H8k9qe7DersxR6KyhUnI
4,Get Low,"['Dillon Francis', 'DJ Snake']",edm,101.033,0.674,0.932,0.7280,3oZoXyU0SkDldgS7AcN4y4
...,...,...,...,...,...,...,...,...
4162,Limba,"['Einmusik', 'Seth Schwarz']",deep house,123.989,0.703,0.732,0.1830,4bGh9ITVoiSTrtLg0h1PQo
4163,Can I,['Britta Arnold'],deep house,120.019,0.809,0.667,0.0325,3qavQewlgidBuYHrJe3Qqb
4164,By Defection,['Dixie Yure'],deep house,124.019,0.927,0.513,0.2420,2auhKxSJCfnsaOVxFoF3uA
4165,Chunky,['Format:B'],deep house,121.989,0.803,0.858,0.8620,5SB73InpxuEQ20qEWhOQFV


In [68]:
sp_more_genres.Genre.value_counts()

house                        413
electro house                392
electronic trap              365
edm                          200
progressive house            200
techno                       200
deep house                   183
bass house                   102
dance                        101
complextro                   101
pop dance                    100
commercial house             100
filter house                 100
latin house                  100
trance                       100
90s eurodance                100
brostep                      100
progressive electro house    100
commercial                   100
indietronica                 100
tropical house               100
electropop                   100
electronica                  100
melbourne bounce              98
big beat                      92
happy hardcore                89
tech house                    89
90s dance                     89
brazilian edm                 88
bass music                    54
alternativ

In [69]:
genres_list = set(sp_more_genres.Genre)
genres_list

{'90s dance',
 '90s eurodance',
 'alternative country',
 'bass house',
 'bass music',
 'big beat',
 'brazilian edm',
 'brostep',
 'commercial',
 'commercial house',
 'complextro',
 'dance',
 'deep house',
 'edm',
 'electro house',
 'electronic trap',
 'electronica',
 'electropop',
 'filter house',
 'happy hardcore',
 'house',
 'indietronica',
 'latin house',
 'melbourne bounce',
 'pop dance',
 'progressive electro house',
 'progressive house',
 'tech house',
 'techno',
 'trance',
 'tropical house'}

In [70]:
genres_to_keep = ['tech house', 'deep house', 'dancefloor dnb', 'progressive trance house',
                 'deep euro house', 'deep vocal house', 'austrian dnb', 'electro swing',
                 'deep liquid bass', 'edm', 'dutch trap', 'modular techno', 'synthpop',
                 'deep pop edm', 'pop house', 'happy hardcore', 'hungarian edm',
                 'french tech house', 'bass music', 'uplifting trance', 'pop edm',
                 'deep hardtechno', 'organic electronic', 'disco', 'disco house', 'acid idm',
                 'old school hard trance', 're:techno', 'dark disco', 'lo-fi house',
                 'deep tropical house', 'latin house', 'electro house,', 'tropical house',
                 'house', 'trance', 'indietronica', 'dark trap', 'metropopolis', 'progressive house',
                 'deep progressive house', 'vapor trap', 'progressive electro house', 'electronica',
                 'big room', 'pop dance', 'electropop', 'electronic trap', '90s eurodance',
                 'brazilian edm', '90s dance', 'filter house', 'complextro', 'commercial house',
                 'dance', 'commercial', ]

In [71]:
more_genres = sp_more_genres.loc[sp_more_genres['Genre'].isin(genres_to_keep)].copy()
more_genres

,Song_Title,Artists,Genre,Tempo,Danceability,Energy,Valence,ID
0,Hero,"['Afrojack', 'David Guetta']",edm,125.001,0.713,0.772,0.5230,29p6UW08MXzwj9thWBW3uF
1,Chasing Stars (feat. James Bay),"['Alesso', 'Marshmello', 'James Bay']",edm,92.871,0.594,0.640,0.4920,6y6xhAgZjvxy5kR5rigpY3
2,Gotta Be You,"['NERVO', 'Carla Monroe']",edm,123.910,0.642,0.869,0.3760,2Q8fFORnsDE6I3RNvSLXSf
3,Okay,"['Nicky Romero', 'MARF', 'Wulf']",edm,124.964,0.513,0.836,0.3040,71H8k9qe7DersxR6KyhUnI
4,Get Low,"['Dillon Francis', 'DJ Snake']",edm,101.033,0.674,0.932,0.7280,3oZoXyU0SkDldgS7AcN4y4
...,...,...,...,...,...,...,...,...
4162,Limba,"['Einmusik', 'Seth Schwarz']",deep house,123.989,0.703,0.732,0.1830,4bGh9ITVoiSTrtLg0h1PQo
4163,Can I,['Britta Arnold'],deep house,120.019,0.809,0.667,0.0325,3qavQewlgidBuYHrJe3Qqb
4164,By Defection,['Dixie Yure'],deep house,124.019,0.927,0.513,0.2420,2auhKxSJCfnsaOVxFoF3uA
4165,Chunky,['Format:B'],deep house,121.989,0.803,0.858,0.8620,5SB73InpxuEQ20qEWhOQFV
